In [ ]:
import csv 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import scipy as sp 
import os 
import scipy.stats as stats 
from scipy.stats import norm 
import matplotlib.mlab as mlab 
from statistics import mean

In [2]:
#high and low separation:

#make lists for new values:
#(1)vhs_norm, (2)vls_norm, (3)hhs_norm, (4)hls_norm - these will have the values of the oscillations around the zero point

#data input:
#vhs, vls, hhs, hls, hji, lji (v: vertical signal, h:horizontal signal) (hs: high stiffness, ls: low stiffness)

def high_and_low(t, ver, hor, mod, thres, vhs, vls, hhs, hls, ths, tls, hji, lji):
    #getting the indices of high stiffness:
    stl = -1
    xl = []
    low_index = []
    for i in mod:
        if i < thres:
            try:
                xl = mod.index(i, stl+1)
            except ValueError:
                break
            else:
                low_index.append(xl)
                stl = xl

    #getting the indices of low stiffness:
    sth = -1
    xh = []
    high_index = []
    for i in mod:
        if i > thres:
            try:
                xh = mod.index(i, sth+1)
            except ValueError:
                break
            else:
                high_index.append(xh)
                sth = xh 
    
    #define signal lists before cuts:
    vhs1 = []
    vls1 = []
    hhs1 = []
    hls1 = []
    ths1 = []
    tls1 = []
    
    #Adding values to time and signal lists:
    for i in high_index:
        vhs1.append(ver[i])
        hhs1.append(hor[i])
        ths1.append(t[i])
        
    for i in low_index:
        vls1.append(ver[i])
        hls1.append(hor[i])
        tls1.append(t[i])
        
    #making sure cut data has full jumps:
    #high:
    for i in range(len(ths1)):
        x = ths1[i]
        y = ths1[i-1]
        if x - y > 4.5 * 10**(-6):
            hji.append(i)
    #low:
    for i in range(len(tls1)):
        x = tls1[i]
        y = tls1[i-1]
        if x - y > 4.5 * 10**(-6):
            lji.append(i)
    
    #cutting data:
    for i in ths1[hji[0]:hji[-1]]:
        ths.append(i)
    for i in vhs1[hji[0]:hji[-1]]:
        vhs.append(i)
    for i in hhs1[hji[0]:hji[-1]]:
        hhs.append(i)
    for i in tls1[lji[0]:lji[-1]]:
        tls.append(i)
    for i in vls1[lji[0]:lji[-1]]:
        vls.append(i)
    for i in hls1[lji[0]:lji[-1]]:
        hls.append(i)

In [3]:
#reject low frequencies/ oscillations

#make lists for new values:
#(1)vhs_norm, (2)vls_norm, (3)hhs_norm, (4)hls_norm - these will have the values of the oscillations around the zero point

#not using butter because that gets rid of quite a lot of the signal
#this takes the average of a jump (where the modulation occured) and subtracts the mean

#data input:
#vhs, vls, hhs, hls, hji, lji

def low_freq_filter(vhs, vls, hhs, hls, hji, lji, vhs_norm, vls_norm, hhs_norm, hls_norm):
    vhs_norm1 = []
    vhs_norm1_lst = []
    vls_norm1 = []
    vls_norm1_lst = []
    hhs_norm1 = []
    hhs_norm1_lst = []
    hls_norm1 = []
    hls_norm1_lst = []

    #vhs
    for i in range(0,len(hji[:-2])):
        x = vhs[hji[i]:hji[i+1]]
        vhs_norm1.append(np.subtract(vhs[hji[i]:hji[i+1]], mean(x)))
                         
    for i in vhs_norm1:
        vhs_norm1_lst.append(i.tolist())
    
    for i in vhs_norm1_lst:
        for j in i:
            vhs_norm.append(j)
     
    #vls
    for i in range(0,len(lji[:-2])):
        x = vls[lji[i]:lji[i+1]]
        vls_norm1.append(np.subtract(vls[lji[i]:lji[i+1]], mean(x)))
                         
    for i in vls_norm1:
        vls_norm1_lst.append(i.tolist())
    
    for i in vls_norm1_lst:
        for j in i:
            vls_norm.append(j)
            
    #hhs
    for i in range(0,len(hji[:-2])):
        x = hhs[hji[i]:hji[i+1]]
        hhs_norm1.append(np.subtract(hhs[hji[i]:hji[i+1]], mean(x)))
                         
    for i in hhs_norm1:
        hhs_norm1_lst.append(i.tolist())
    
    for i in hhs_norm1_lst:
        for j in i:
            hhs_norm.append(j)
     
    #hls
    for i in range(0,len(lji[:-2])):
        x = hls[lji[i]:lji[i+1]]
        hls_norm1.append(np.subtract(hls[lji[i]:lji[i+1]], mean(x)))
                         
    for i in hls_norm1:
        hls_norm1_lst.append(i.tolist())
    
    for i in hls_norm1_lst:
        for j in i:
            hls_norm.append(j)

In [4]:
#FFT function
#inputs: vhs_norm, vls_norm, hhs_norm, hls_norm + f_s sampling rate

#need to define empty lists for PSD data: (1) psd_vhs, (2) psd_vls, (3) psd_hhs, (4) psd_hls

def FFTviaPSD(vhs_norm, vls_norm, hhs_norm, hls_norm, psd_vhs, psd_vls, psd_hhs, psd_hls, f_s, figname):
    plt.figure(figsize=(15,7))
    
    vtot = []
    htot = []
    
    for i in vhs_norm:
        vtot.append(i)
    for i in vls_norm:
        vtot.append(i)
        
    for i in hhs_norm:
        htot.append(i)
    for i in hls_norm:
        htot.append(i)

    vh_psd_plt = plt.psd(vhs_norm, Fs=f_s, label='vertical - high stiffness')
    vl_psd_plt = plt.psd(vls_norm, Fs=f_s, label='vertical - low stiffness')
    v_psd_plt = plt.psd(vtot, Fs=f_s, color='black', alpha=0.5)

    hh_psd_plt = plt.psd(hhs_norm, Fs=f_s, label='horizontal - high stiffness')
    hl_psd_plt = plt.psd(hls_norm, Fs=f_s, label='horizontal - low stiffness')
    h_psd_plt = plt.psd(htot, Fs=f_s, color='black', alpha=0.5, label='totals')

    plt.legend()

    plt.savefig(figname, dpi=600)
    
    plt.show()
    
    
    
    for i in vh_psd_plt:
        psd_vhs.append(vh_psd_plt)
        
    for i in vl_psd_plt:
        psd_vls.append(vl_psd_plt)
        
    for i in hh_psd_plt:
        psd_hhs.append(hh_psd_plt)
        
    for i in hl_psd_plt:
        psd_hls.append(hl_psd_plt)

In [ ]:
#getting the frequencies from the PSD:

def frequencies(psd_vhs, psd_vls, psd_hhs, psd_hls, vhP, vhf, vlP, vlf, hhP, hhf, hlP, hlf):
    #first get list of power and frequency values, reject the first two points where it peaks at frequency=0
    vh_psd_P = psd_vhs[1][0].tolist()
    vh_psd_f = psd_vhs[1][1].tolist()
    vl_psd_P = psd_vls[1][0].tolist()
    vl_psd_f = psd_vls[1][1].tolist()

    hh_psd_P = psd_hhs[1][0].tolist()
    hh_psd_f = psd_hhs[1][1].tolist()
    hl_psd_P = psd_hls[1][0].tolist()
    hl_psd_f = psd_hls[1][1].tolist()
    
    for i in vh_psd_P:
        vhP.append(i)
    for i in vh_psd_f:
        vhf.append(i)
    for i in vl_psd_P:
        vlP.append(i)
    for i in vl_psd_f:
        vlf.append(i)
    for i in hh_psd_P:
        hhP.append(i)
    for i in hh_psd_f:
        hhf.append(i)
    for i in hl_psd_P:
        hlP.append(i)
    for i in hl_psd_f:
        hlf.append(i)